In [ ]:
!ollama pull llama3.2

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama
MODEL = "llama3.2"

In [ ]:
#test

messages = [
    {"role": "system", "content": "You are a snarky assistant."},
    {"role": "user", "content": "what is 1+1"}
]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = ollama.chat(
  model=MODEL, messages=messages) 
print(response['message']['content'])

In [ ]:
#website scraping

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ai_blog = Website("https://allensthoughts.com/")
print(ai_blog.title)

In [ ]:
system_prompt =   "You are a highly accurate and detail-oriented summarization assistant. \
    You read and understand web page content and generate clear, concise, and factually correct summaries. \
    Your summaries preserve key information, context, and structure while removing irrelevant details or marketing fluff. \
    Use professional, neutral language. Focus on the purpose of the website, main offerings, audience, and any unique features. \
    If the website is an article or blog post, summarize the main argument, key points, and conclusion. \
    Only summarize what is visible in the content provided. Do not hallucinate or assume extra information."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(ai_blog)

In [ ]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

In [ ]:
summarize("https://allensthoughts.com/2025/03/31/ais-real-revolution-is-just-beginning/")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://allensthoughts.com/2025/03/31/ais-real-revolution-is-just-beginning/")